# Générateur de fiche de paye Pajemploi

Vous pouvez vous aider de ce générateur pour remplir les fiches de paye de votre auxiliare parentale sur ([Pajemploi](https://www.pajemploi.urssaf.fr/)) directement en ligne.

Remplissez la configuration ci-dessous pour le contrat qui vous lie avec l'auxiliaire parentale (salaire horaire, etc.) et les spécificités du mois pour lequel vous voulez générer la fiche de paye.

Puis, cliquez en haut de la page sur `Run` puis `Run All Cells`.

## Setup

In [ ]:
from datetime import datetime
import utils

## Configuration (entrez vos données)

La configuration à éditer ci-dessous représente les détails du contrat de votre auxiliaire parentale.

Schéma de données:
- `start_date_period` : date de départ de la période pour laquelle ce contrat s'applique.
- `end_date_period` : date de fin de la période pour laquelle ce contrat s'applique.
- `hourly_salary` : salaire horaire **NET**.
- `worked_weekly_hours` : nombre total d'heures hebdomadaires travaillées (max 50 légalement).
- `recovered_weekly_hours` : nombre d'heures hebdomadaires "récupérées" en jours non-travaillés.
- `transport_monthly_cost` : frais de transport mensuels de l'auxiliaire parentale.
- `transport_allowance_prct` : part des frais de transport remboursés par les employeurs (usuellement 100%).
- `daily_meal_allowance` : indemnité de repas quotidienne (par jour travaillé).
- `allocation_key` : clé de répartition des frais sur les différentes famille en cas de garde partagée.

Le schéma de données permet de donner plusieurs configs dans le cas ou un changement de contrat a eu lieu en cours de mois. Dans ce cas, faîtes bien attention à ce que les périodes temporelles soient bien disjointes.

In [ ]:
configs = [
    {
        "start_date_period": "2023-06-01",
        "end_date_period": "2023-06-30",
        "config":{
            "hourly_salary": 10.25,
            "worked_weekly_hours": 50,
            "recovered_weekly_hours": 0,
            "transport_monthly_cost": 84.10,
            "transport_allowance_prct": 1,
            "daily_meal_allowance": 5,
            "allocation_key": {
                "family": ["Martin", "Dupont"],
                "salary": [0.6, 0.4],
                "meal_allowance": [0.5, 0.5],
                "transport_allowance": [0.5, 0.5]
            }
        }
    }
]

La cellule ci-dessous permet de spécifier les absences de l'auxiliaire parentale pendand le mois par catégorie: vacances, jours fériés et congés maladies.

- `vacation_calendar` : liste des périodes de congés payés et/ou jours de récupération sur le mois.
    - `start_date_included` : premier jour de vacance de la période
    - `end_date_excluded` : premier jour de **retour au travail** de l'auxiliaire parentale
    - **Attention** : Pour signifier que l'auxiliaire parentale a été absente une demi-journée seulement, inscriver la même date pour `start_date_included` et `end_date_excluded`.
- `holidays` : liste des jours fériés sur la période.
- `sick_days` : liste des périodes de congés maladie sur le mois.
    - `start_date_included` : premier jour de vacance de la période
    - `end_date_excluded` : premier jour de **retour au travail** de l'auxiliaire parentale
    - **Attention** : Pour signifier que l'auxiliaire parentale a été absente une demi-journée seulement, inscriver la même date pour `start_date_included` et `end_date_excluded`.

Si vous n'avez pas d'absence à signaler ce mois-ci dans l'une ou l'autre catégorie, laisser la liste vide : `[]`

In [ ]:
vacation_calendar =  [
    {
        "start_date_included": "2023-04-14",
        "end_date_excluded":"2023-04-15"
    }
]
holidays = [
    {
        "date": "2023-04-10",
        "name": "Pâques"
    }
]
sick_days = [
    {
        "start_date_included": "2023-05-29",
        "end_date_excluded":"2023-04-15"
    }
]

## Revue du mois

In [ ]:
start_date = datetime.strptime(configs[0]['start_date_period'], "%Y-%m-%d").date()
end_date = datetime.strptime(configs[len(configs)-1]['end_date_period'], "%Y-%m-%d").date()

In [ ]:
df = utils.display_monthly_review(
    start_date=start_date,
    end_date=end_date, 
    vacation_calendar=vacation_calendar, 
    holidays=holidays, 
    sick_days=sick_days
)

## Déclaration Pajemploi

In [ ]:
period_wages, cp_aquired, recovered_days_acquired = utils.compute_period_wages(start_date, end_date, configs, vacation_calendar=vacation_calendar, holidays=holidays, sick_days=sick_days)

In [ ]:
total_wages = utils.compute_total_wages(period_wages)
total_wages

In [ ]:
utils.display_urssaf_input_values(total_wages, 'Martin')

In [ ]:
utils.display_paid_leave_acquired(cp_aquired, recovered_days_acquired)